In [1]:
import pandas as pd

from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances

import torch

from mcce import MCCE

data_name = "adult"
# data_name = 'give_me_some_credit'
data_name = 'compas'
K = 1000
n_test = 100
seed = 1

dataset = OnlineCatalog(data_name)

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/nr/samba/

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
torch.manual_seed(0)
ml_model = MLModelCatalog(
        dataset, 
        model_type="ann", 
        load_online=False, 
        backend="pytorch"
    )

if data_name == 'adult':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=1024,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'give_me_some_credit':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=2048,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'compas':
    ml_model.train(
    learning_rate=0.002,
    epochs=25,
    batch_size=25,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:n_test]

balance on test set 0.8131345863037374, balance on test set 0.8191834089436163
Epoch 0/24
----------


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


train Loss: 0.5018 Acc: 0.7280

test Loss: 0.4022 Acc: 0.8192

Epoch 1/24
----------
train Loss: 0.4002 Acc: 0.8283

test Loss: 0.3704 Acc: 0.8471

Epoch 2/24
----------
train Loss: 0.3796 Acc: 0.8356

test Loss: 0.3557 Acc: 0.8419

Epoch 3/24
----------
train Loss: 0.3665 Acc: 0.8423

test Loss: 0.3405 Acc: 0.8477

Epoch 4/24
----------
train Loss: 0.3620 Acc: 0.8438

test Loss: 0.3483 Acc: 0.8464

Epoch 5/24
----------
train Loss: 0.3589 Acc: 0.8445

test Loss: 0.3409 Acc: 0.8542

Epoch 6/24
----------
train Loss: 0.3592 Acc: 0.8434

test Loss: 0.3361 Acc: 0.8555

Epoch 7/24
----------
train Loss: 0.3579 Acc: 0.8451

test Loss: 0.3358 Acc: 0.8529

Epoch 8/24
----------
train Loss: 0.3564 Acc: 0.8445

test Loss: 0.3367 Acc: 0.8548

Epoch 9/24
----------
train Loss: 0.3559 Acc: 0.8440

test Loss: 0.3383 Acc: 0.8516

Epoch 10/24
----------
train Loss: 0.3564 Acc: 0.8471

test Loss: 0.3534 Acc: 0.8542

Epoch 11/24
----------
train Loss: 0.3556 Acc: 0.8447

test Loss: 0.3376 Acc: 0.8535



In [3]:
1 - factuals.shape[0] / dataset.df.shape[0]

0.8791315618924174

In [4]:
y_col = dataset.target
cont_feat = dataset.continuous

cat_feat = dataset.categorical
cat_feat_encoded = dataset.encoder.get_feature_names(dataset.categorical)

if data_name == 'adult': 
    fixed_features = ['age', 'sex_Male']
elif data_name == 'give_me_some_credit':
    fixed_features = ['age']
elif data_name == 'compas':
    fixed_features = ['age', 'sex_Male', 'race_Other']

#  Create dtypes for MCCE()
dtypes = dict([(x, "float") for x in cont_feat])
for x in cat_feat_encoded:
    dtypes[x] = "category"
df = (dataset.df).astype(dtypes)

In [6]:
results = []

import time
start = time.time()
time1 = time.time()
# fixed_features = names in dataset
# categorical = original feature names

mcce = MCCE(fixed_features=fixed_features, continuous=dataset.continuous, categorical=dataset.categorical,\
            model=ml_model, seed=1, catalog=dataset.catalog)

mcce.fit(df.drop(y_col, axis=1), dtypes)

time2 = time.time()
print(time2 - time1) # 0.24

synth_df = mcce.generate(test_factual.drop(y_col, axis=1), k=K)

time3 = time.time()
print(time3 - time2) # 2.41

mcce.postprocess(data=df, synth=synth_df, test=test_factual, response=y_col, \
    inverse_transform=dataset.inverse_transform, cutoff=0.5)

time4 = time.time()
print(time4 - time3) # 

timing = time.time() - start

mcce.results_sparse['time (seconds)'] = timing

results.append([mcce.results_sparse.L0.mean(), mcce.results_sparse.L2.mean(), mcce.results_sparse.feasibility.mean(),\
     mcce.results_sparse.violation.mean(), mcce.results_sparse.shape[0], timing])

0.236433744430542
0.26670312881469727


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


74.08056855201721


In [7]:
# mcce.results_sparse.to_csv(f"Results/{data_name}_mcce_results_k_{K}_n_{n_test}.csv")

In [8]:
# data=df
# synth=synth_df
# test=test_factual
# response=y_col
# inverse_transform=dataset.inverse_transform
# cutoff=0.5

# # Predict response of generated data
# synth[response] = mcce.model.predict(synth)
# synth_positive = synth[synth[response]>=cutoff] # drop negative responses

# # Duplicate original test observations N times where N is number of positive counterfactuals
# n_counterfactuals = synth_positive.groupby(synth_positive.index).size()
# n_counterfactuals = pd.DataFrame(n_counterfactuals, columns = ['N'])

# test_repeated = test.copy()

# test_repeated = test_repeated.join(n_counterfactuals)
# test_repeated.dropna(inplace = True)

# test_repeated = test_repeated.reindex(test_repeated.index.repeat(test_repeated.N))
# test_repeated.drop(['N'], axis=1, inplace=True)

# mcce.test_repeated = test_repeated

In [9]:
# synth=synth_positive
# test=mcce.test_repeated
# data=data
# model=mcce.model
# response=response
# inverse_transform=inverse_transform

In [10]:
# import metrics

# features = synth.columns.to_list()
# features.remove(response)
# synth.sort_index(inplace=True)

# if inverse_transform:  # necessary for violation rate
#     df_decoded_cfs = inverse_transform(synth.copy())
#     df_decoded_factuals = inverse_transform(test.copy())

# else:
#     df_decoded_cfs = synth.copy()
#     df_decoded_factuals = test.copy()


# synth_metrics = synth.copy()

# # 1) Distance: Sparsity and Euclidean distance
# factual = test[features]#.sort_index().to_numpy()
# counterfactuals = synth[features]#.sort_index().to_numpy()

# time1 = time.time()
# distances = pd.DataFrame(metrics.distance(counterfactuals, factual, mcce.model), index=factual.index)

# time2 = time.time()
# mcce.distance_cpu_time = time2 - time1
# synth_metrics = pd.concat([synth_metrics, distances], axis=1)

In [11]:
# counterfactuals.shape

In [12]:
# cols = data.columns.to_list()
# cols.remove(response)

# time1 = time.time()
# synth_metrics['feasibility'] = metrics.feasibility(counterfactuals, factual, cols, response)

# time2 = time.time()
# mcce.feasibility_cpu_time = time2 - time1

# # 3) Success
# synth_metrics['success'] = 1

In [13]:
# time1 = time.time()
# violations = metrics.constraint_violation(df_decoded_cfs, df_decoded_factuals, \
#     mcce.continuous, mcce.categorical, mcce.catalog['immutable'])

# synth_metrics['violation'] = violations
# time2 = time.time()
# mcce.violation_cpu_time = time2 - time1


In [14]:
# results2 = pd.DataFrame(results, columns=['L0', 'L2', 'feasibility', 'violation', 'NCE', 'timing'])